# Data Extraction Notebook
Christopher Marais
gmarais@ufl.edu
2023/12/14

This notebook aims to extract the raw data inot a CSV format. THe raw data is in a python class format that has been saved as a binary object using pickle. 
The python class is described with detail in the multirecording_spikeanalysis.py python script. 


`/01_Raw_data/phase2_collection.pkl` → `</03_Scripts/00_Data_Extraction.ipynb>` → `/02_Clean_data/00_recording_event_times_labels.csv`

This notebook should take approximately 5-10 minutes to run. 


### Import necessary Python packages

In [3]:
import os
import pickle
import multirecording_spikeanalysis as spike
import numpy as np
# import pandas as pd # use pandas for more functionality
import modin.pandas as pd # use modin to speed things up

### Define working directory relative to repository

In [4]:
# get working directory as paerent directory of current directory
cwd = os.getcwd()
pwd = os.path.dirname(cwd)

### Import .pkl files

In [5]:
# import pickle files given by lab
with open(pwd + '/01_Raw_data/phase2_collection.pkl', 'rb') as f:
    phase2 = pickle.load(f)

# Not using phase 3 data
# with open(pwd + '/01_Raw_data/phase3_collection.pkl', 'rb') as f:
#     phase3 = pickle.load(f)

### Rework Data from .pkl files

#### Behavioral labels and event ranges dataframe

In [7]:
# create extracted labels dataframe
extracted_labels_df = pd.DataFrame() # create empty dataframe
temp_df_lst = [] # create empty list to store dataframes
for i, j in phase2.collection.items(): # loop through each recording
    temp_df = pd.DataFrame() # create empty dataframe
    start_time_lst = [] # create empty list to store start times
    end_time_lst = [] # create empty list to store end times
    behavior_lab_lst = [] # create empty list to store behavior labels
    for k, v in j.event_dict.items(): # loop through each behavior
        start_time_lst += list(v[:,0]) # add start times to list
        end_time_lst += list(v[:,1]) # add end times to list
        behavior_lab_lst += list([k] * len(v)) # add behavior labels to list
    temp_df['behavior_label'] = behavior_lab_lst # add behavior labels to dataframe
    temp_df['start_time'] = start_time_lst # add start times to dataframe
    temp_df['end_time'] = end_time_lst # add end times to dataframe
    temp_df['collection_key'] = i # add recording name to dataframe
    temp_df['subject'] = j.subject # add subject to dataframe
    temp_df_lst.append(temp_df) # add dataframe to list
extracted_labels_df = pd.concat(temp_df_lst) # concatenate all dataframes in list to one dataframe

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Perhaps you already have a cluster running?
Hosting the HTTP server on port 59396 instead
Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [8]:
# split the order labels from the behaviour labels to a new column
# Get a dataframe that only includes the behaviour labels
order_df = extracted_labels_df[extracted_labels_df['behavior_label'].isin([
    'exposure 1',
    'exposure 2', 
    'exposure 3'])].reset_index(drop=True)
order_df = order_df.rename(columns={'behavior_label': 'order'}) # change order label column name

# merge dataframes on all columns except the order and behaviour columns
merged_order_df = pd.merge(
    order_df, 
    extracted_labels_df, 
    on=[
        'start_time',
        'end_time',
        'collection_key',
        'subject'],
    how='right')

extracted_labels_df = merged_order_df # rename dataframe

### Units dataframe & unit timestamps list
Save a dataframe of the units used for each subject at each recording session.

Extract the timestamps for each event for each unit from the data.

Merge labels dataframes into a single long dataframe. So that data can be easily related to each recording session.

In [9]:
# create metadata dataframe
data_df = pd.DataFrame()
temp_df_lst = []
for i, j in phase2.collection.items():
    temp_df = pd.DataFrame()
    temp_df['units'] = j.unit_timestamps.keys() # add neurons/units to dataframe
    temp_df['timestamps'] = j.unit_timestamps.values() # add neurons/units to dataframe
    temp_df['collection_key'] = i # add recording name to dataframe
    temp_df['subject'] = j.subject # add subject to dataframe
    temp_df_lst.append(temp_df) # add dataframe to list
data_df = pd.concat(temp_df_lst) # concatenate all dataframes in list to one dataframe

In [10]:
# Merge dataframes
merged_data_df = pd.merge(extracted_labels_df, data_df, on=['collection_key', 'subject'])

In [11]:
# Control home much before and after event to include with event.
BEFORE_EVENT_BUFFER = 1
AFTER_EVENT_BUFFER = 0
BEFORE_EVENT_BUFFER = BEFORE_EVENT_BUFFER*20000
AFTER_EVENT_BUFFER = AFTER_EVENT_BUFFER*20000

In [12]:
# get list of timestamps for each unit
timestamps_lst = merged_data_df['timestamps'].tolist()
# Find the length of the longest array
max_length = max(arr.size for arr in timestamps_lst)
# make a nan array with the same max shape as the array of timestamps
timestamps_array = np.full((40866, max_length), np.nan)
# Fill in the nan_filled_array with values from timestamps
for i, arr in enumerate(timestamps_lst):
    timestamps_array[i, :arr.size] = arr

In [13]:
# get the start and end times for each behavior into arrays
min_thresholds = np.array(merged_data_df['start_time']) - BEFORE_EVENT_BUFFER # subtract 20 000 for one extra second before event
max_thresholds = np.array(merged_data_df['end_time']) + AFTER_EVENT_BUFFER
# Reshape the threshold arrays to column vectors for broadcasting
min_thresholds = min_thresholds[:, np.newaxis]
max_thresholds = max_thresholds[:, np.newaxis]
# Apply thresholds using broadcasting
lower_mask = timestamps_array < min_thresholds
upper_mask = timestamps_array > max_thresholds
# Replace values that are either too low or too high with NaN
timestamps_array[lower_mask | upper_mask] = np.nan

In [14]:
# import pandas as pd # use pandas if modin doesn't work (should be done automatically)
# remove nan values from arrays in list
event_ts_lst = list(timestamps_array) # convert array to list
event_ts_lst = [arr[~np.isnan(arr)] for arr in event_ts_lst] # remove nan values from arrays in list
merged_data_df['event_timestamps'] = event_ts_lst # add event timestamps to dataframe
# remove all duplicate rows
merged_data_df = merged_data_df.drop_duplicates().reset_index(drop=True)
# drop all rwos that have exposure 1, 2, or 3 as the behaviour label
merged_data_df = merged_data_df[~merged_data_df['behavior_label'].isin([
    'exposure 1',
    'exposure 2',
    'exposure 3'])].reset_index(drop=True)
# Convert each float in the lists to an integer
merged_data_df['event_timestamps'] = merged_data_df['event_timestamps'].apply(lambda lst: [int(x) for x in lst])

### Save output data

In [19]:
# save dataframe as csv file
merged_data_df[['order','start_time','end_time','collection_key','subject','behavior_label','units']].to_csv(pwd + "/02_Clean_data/00_recording_event_times_labels.csv", index=False)

# save list of arrays to pkl files
with open(pwd + "/02_Clean_data/00_timestamps_arrays_list.pkl", 'wb') as file:
    pickle.dump(merged_data_df['timestamps'].tolist(), file)
with open(pwd + "/02_Clean_data/00_event_timestamps_arrays_list.pkl", 'wb') as file:
    pickle.dump(merged_data_df['event_timestamps'].tolist(), file)
# np.save(pwd + "/02_Clean_data/00_event_timestamps.pkl", dtw_distance_matrix)

In [17]:
merged_data_df

,order,start_time,end_time,collection_key,subject,behavior_label,units,timestamps,event_timestamps
0,NaN,15167.0,24733.0,20230803_101331_1_merged.rec,1.1,acquisition,2,"[133, 359, 761, 841, 1042, 1142, 1310, 1398, 1...","[133, 359, 761, 841, 1042, 1142, 1310, 1398, 1..."
1,NaN,15167.0,24733.0,20230803_101331_1_merged.rec,1.1,acquisition,26,"[149, 2407, 2955, 6394, 6924, 11713, 12780, 13...","[149, 2407, 2955, 6394, 6924, 11713, 12780, 13..."
2,NaN,15167.0,24733.0,20230803_101331_1_merged.rec,1.1,acquisition,196,"[394, 1740, 2021, 2752, 3636, 5356, 5854, 6371...","[394, 1740, 2021, 2752, 3636, 5356, 5854, 6371..."
3,NaN,15167.0,24733.0,20230803_101331_1_merged.rec,1.1,acquisition,113,"[436, 7630, 9762, 11461, 13316, 14145, 21222, ...","[436, 7630, 9762, 11461, 13316, 14145, 21222, ..."
4,NaN,15167.0,24733.0,20230803_101331_1_merged.rec,1.1,acquisition,91,"[449, 4067, 11824, 22087, 32790, 53466, 55978,...","[449, 4067, 11824, 22087]"
...,...,...,...,...,...,...,...,...,...
25593,exposure 3,1985700.0,1986200.0,20230818_133620_1_merged.rec,1.4,novel,73,"[352534, 1489090, 3711465, 3936724, 4482099, 6...",[]
25594,exposure 3,1985700.0,1986200.0,20230818_133620_1_merged.rec,1.4,novel,141,"[413994, 636033, 764028, 1171529, 1250682, 149...",[]
25595,exposure 3,1985700.0,1986200.0,20230818_133620_1_merged.rec,1.4,novel,36,"[449953, 451680, 455769, 458347, 462015, 48572...",[]
25596,exposure 3,1985700.0,1986200.0,20230818_133620_1_merged.rec,1.4,novel,95,"[472131, 492784, 527617, 761281, 788850, 93311...",[]
